In [ ]:
######################################################################################################################
######################################################################################################################
#
# TRAINING
#
######################################################################################################################
######################################################################################################################

In [266]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt

In [267]:
# importing the data
tm_dummy = pd.read_csv('data/train_store_states_Python.csv')

In [268]:
tm_dummy = tm_dummy[tm_dummy['Sales'] != 0]

In [269]:
x = tm_dummy.drop(['Sales', 'Customers', 'Date', 'Unnamed: 0'], 1)
y = tm_dummy[['Store','Sales']]

In [270]:
x['PromoInterval'].fillna(0, inplace = True)
x.ix[(x['PromoInterval'] == 'Jan,Apr,Jul,Oct'), 'PromoInterval'] = 1
x.ix[(x['PromoInterval'] == 'Feb,May,Aug,Nov'), 'PromoInterval'] = 2
x.ix[(x['PromoInterval'] == 'Mar,Jun,Sept,Dec'), 'PromoInterval'] = 3

In [271]:
x_PI = pd.get_dummies(x.PromoInterval, 'PI')
x = pd.concat([x, x_PI[x_PI.columns[1:]]],axis = 1)

In [272]:
x.drop(['Promo2SinceWeek','Promo2SinceYear','PromoInterval','CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear'],1,inplace = True)

In [289]:
runs = 1115
y = tm_dummy[['Store','Sales']]

models = {}
print "Start!"
for i in range(1,runs+1):
    model = linear_model.LinearRegression()
    x_temp = x[x.Store==i].drop('Store',1)
    y_temp = np.log(y[y.Store==i].drop('Store',1))
    models['model_' + str(i)] = model.fit(x_temp, y_temp)
print "End!"

Start!
End!


In [233]:
def rmse(predictions, targets):
   return ((predictions - targets)/targets) ** 2

In [234]:
scores = {}
y = tm_dummy[['Store','Sales', 'month']]
print "Start!"
for i in range(1,runs):
    x_temp = x[(x.Store == i) & ((x.month == 8) | (x.month == 9))].drop('Store',1)
    trgts = np.array(y[(x.Store == i) & ((x.month == 8) | (x.month == 9))].drop(['Store','month'],1))
    preds = np.exp(models['model_' + str(i)].predict(x_temp))
    scores['model_' + str(i)] = rmse(preds, trgts)
print "End!"

Start!
End!


In [235]:
total = 0
for i in range(1,runs):
    total += sum(scores['model_' + str(i)])
np.sqrt(total/(runs*104))[0]

0.17850673642449896

In [ ]:
######################################################################################################################
######################################################################################################################
#
# TESTING
#
######################################################################################################################
######################################################################################################################

In [260]:
test = pd.read_csv('data/test_store_states_Python.csv')

In [278]:
x_test = test.drop(['Date', 'Unnamed: 0'], 1)
x_test['PromoInterval'].fillna(0, inplace = True)
x_test.ix[(x_test['PromoInterval'] == 'Jan,Apr,Jul,Oct'), 'PromoInterval'] = 1
x_test.ix[(x_test['PromoInterval'] == 'Feb,May,Aug,Nov'), 'PromoInterval'] = 2
x_test.ix[(x_test['PromoInterval'] == 'Mar,Jun,Sept,Dec'), 'PromoInterval'] = 3
x_PI = pd.get_dummies(x_test.PromoInterval, 'PI')
x_test = pd.concat([x_test, x_PI[x_PI.columns[1:]]],axis = 1)
x_test.drop(['Promo2SinceWeek','Promo2SinceYear','PromoInterval','CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear'],1,inplace = True)

In [282]:
x_test['State_BW'] = 0
x_test['State_SN'] = 0
x_test['State_ST'] = 0
x_test['State_TH'] = 0
x_test['StateHoliday_b'] = 0
x_test['StateHoliday_c'] = 0

In [303]:
preds = []
y = tm_dummy[['Store','Sales', 'month']]
print "Start!"
for i in x_test['Store'].unique().astype(int):
    x_temp = x_test[(x_test.Store == i)].drop(['Id','Store'],1)
    pred = np.exp(models['model_' + str(i)].predict(x_temp))
    for j in pred:
        preds.append(j[0])
print "End!"

Start!
End!


In [308]:
x_test['Sales'] = preds

In [311]:
x_test.ix[(x_test['Open'] == 0), 'Sales'] = 0

In [314]:
submit = x_test[['Id','Sales']]
submit = submit.sort(columns = ['Id'], ascending = True)

In [316]:
submit.to_csv("data/Model1_submit.csv")